In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]

In [10]:
from sklearn.model_selection  import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import ComplementNB

param_grid={'alpha':[1.0,5,10],'force_alpha':[True,False],'fit_prior':[True,False],'norm':[True,False]}
grid=GridSearchCV(ComplementNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [1.0, 5, 10], 'fit_prior': [True, False],
                         'force_alpha': [True, False], 'norm': [True, False]},
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [13]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'alpha': 1.0, 'fit_prior': True, 'force_alpha': True, 'norm': False}: 0.5371480765097786


In [14]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[42 37]
 [20 21]]


In [15]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.68      0.53      0.60        79
           1       0.36      0.51      0.42        41

    accuracy                           0.53       120
   macro avg       0.52      0.52      0.51       120
weighted avg       0.57      0.52      0.54       120



In [16]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.5152824945970979

In [17]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_force_alpha,param_norm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.017721,0.004193,0.021680,0.008276,1.0,True,True,True,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.187970,0.187970,0.187970,0.204545,0.204545,0.194600,0.008120,13
1,0.010628,0.003483,0.014057,0.002472,1.0,True,True,False,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,1
2,0.012143,0.003788,0.013563,0.002560,1.0,True,False,True,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.187970,0.187970,0.187970,0.204545,0.204545,0.194600,0.008120,13
3,0.009916,0.001634,0.012702,0.001158,1.0,True,False,False,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,1
4,0.010361,0.003715,0.011464,0.004846,1.0,False,True,True,"{'alpha': 1.0, 'fit_prior': False, 'force_alph...",0.187970,0.187970,0.187970,0.204545,0.204545,0.194600,0.008120,13
5,0.032169,0.029408,0.012822,0.003618,1.0,False,True,False,"{'alpha': 1.0, 'fit_prior': False, 'force_alph...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,1
6,0.007945,0.003420,0.013813,0.004641,1.0,False,False,True,"{'alpha': 1.0, 'fit_prior': False, 'force_alph...",0.187970,0.187970,0.187970,0.204545,0.204545,0.194600,0.008120,13
7,0.012025,0.006973,0.014091,0.004080,1.0,False,False,False,"{'alpha': 1.0, 'fit_prior': False, 'force_alph...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,1
8,0.006928,0.003401,0.016836,0.002680,5,True,True,True,"{'alpha': 5, 'fit_prior': True, 'force_alpha':...",0.187970,0.187970,0.187970,0.204545,0.204545,0.194600,0.008120,13
9,0.015134,0.009638,0.011699,0.003705,5,True,True,False,"{'alpha': 5, 'fit_prior': True, 'force_alpha':...",0.507692,0.488602,0.540136,0.578065,0.541831,0.531265,0.030843,5
